In [ ]:
import random
from Crypto.Util import number
from math import gcd
import hashlib

class RabinCryptosystem:
    @staticmethod
    def generate_key(bit_length):
        def blum_prime(bit_length):
            while True:
                p = number.getPrime(bit_length)
                if p % 4 == 3:
                    return p

        p = blum_prime(bit_length // 2)
        q = blum_prime(bit_length // 2)
        n = p * q
        return n, p, q

    @staticmethod
    def encrypt_rabin(m, n):
        return pow(m, 2, n)

    @staticmethod
    def decrypt_rabin(c, p, q):
        def extended_gcd(a, b):
            if b == 0:
                return a, 1, 0
            gcd, x1, y1 = extended_gcd(b, a % b)
            x = y1
            y = x1 - (a // b) * y1
            return gcd, x, y

        n = p * q

        # Compute the square roots modulo p and q
        mp = pow(c, (p + 1) // 4, p)
        mq = pow(c, (q + 1) // 4, q)

        # Use the extended Euclidean algorithm to find yp and yq
        gcd, yp, yq = extended_gcd(p, q)

        # Compute the four possible plaintexts
        r1 = (yp * p * mq + yq * q * mp) % n
        r2 = (yp * p * mq - yq * q * mp) % n
        r3 = n - r1
        r4 = n - r2

        return [r1, r2, r3, r4]

class AffineCipher:
    @staticmethod
    def mod_inverse(a, m):
        a = a % m
        for x in range(1, m):
            if (a * x) % m == 1:
                return x
        return None

    @staticmethod
    def encrypt_affine(text, a, b):
        if gcd(a, 26) != 1:
            raise ValueError("Key 'a' must be coprime with 26.")
        cipher = ''
        for char in text:
            if char.isalpha():
                offset = 65 if char.isupper() else 97
                cipher += chr(((a * (ord(char) - offset) + b) % 26) + offset)
            else:
                cipher += char
        return cipher

    @staticmethod
    def decrypt_affine(cipher, a, b):
        a_inv = AffineCipher.mod_inverse(a, 26)
        if a_inv is None:
            raise ValueError("Modular inverse for 'a' does not exist.")
        text = ''
        for char in cipher:
            if char.isalpha():
                offset = 65 if char.isupper() else 97
                text += chr(((a_inv * ((ord(char) - offset) - b)) % 26) + offset)
            else:
                text += char
        return text

class Hashing:
    @staticmethod
    def hash_message(message, key):
        # Using SHA-512 for hashing
        return hashlib.sha512((key + message).encode('ascii')).hexdigest()

if __name__ == "__main__":
    bit_length = 512
    n, p, q = RabinCryptosystem.generate_key(bit_length)

    # Take input from user for Affine Cipher keys
    print("Affine Cipher Encryption")
    a = int(input("Enter key 'a' (must be coprime with 26): "))
    b = int(input("Enter key 'b': "))
    secret_key = input("Enter a secret key for hashing: ")

    # Input message
    message = input("Enter the message to encrypt: ")
    print(f"Message sent by sender: {message}")

    # Step 1: Hash the message
    hashed_message = Hashing.hash_message(message, secret_key)
    print(f"Hashed Message (SHA-512): {hashed_message}")

    # Step 2: Encrypt with Affine Cipher
    affine_encrypted = AffineCipher.encrypt_affine(message, a, b)
    print(f"Affine Encrypted Message: {affine_encrypted}")

    # Step 3: Convert Affine Cipher output to integer and encrypt with Rabin
    m = int.from_bytes(affine_encrypted.encode('ascii'), byteorder='big')
    rabin_encrypted = RabinCryptosystem.encrypt_rabin(m, n)
    print(f"Rabin Encrypted Message: {rabin_encrypted}")

    # Step 4: Decrypt with Rabin Cryptosystem
    rabin_decrypted_candidates = RabinCryptosystem.decrypt_rabin(rabin_encrypted, p, q)

    final_affine_message = None
    for candidate in rabin_decrypted_candidates:
        try:
            candidate_message = candidate.to_bytes((candidate.bit_length() + 7) // 8, byteorder='big').decode('ascii')
            # Step 5: Decrypt with Affine Cipher
            decrypted_message = AffineCipher.decrypt_affine(candidate_message, a, b)
            if Hashing.hash_message(decrypted_message, secret_key) == hashed_message:
                final_affine_message = decrypted_message
                break
        except (ValueError, OverflowError):
            continue

    print(f"Message received by Receiver(Decrypt): {final_affine_message}")



Affine Cipher Encryption
Enter key 'a' (must be coprime with 26): 7
Enter key 'b': 11
Enter a secret key for hashing: a
Enter the message to encrypt: Hello
Message sent by sender: Hello
Hashed Message (SHA-512): ed830d14ebab50cd573d712ae50c6f5d6315d172c2540763797ef09538144bc24f71b6d2e8981badd058e0aa58db8ad654cfeb56586af54410f36636d2e62a82
Affine Encrypted Message: Inkkf
Rabin Encrypted Message: 99467790452426158533796
Message received by Receiver: Hello


In [ ]:
!pip install pycryptodome


   ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 2.3/2.3 MB 22.7 MB/s eta 0:00:00
